## Introduction

In this lab assessment, you'll practice your knowledge of JOIN statements and subqueries, using various types of joins and various methods for specifying the links between them. One of the main benefits of using a relational database is the table relations that define them which allow you to access and connect data together via shared columns. By writing more advanced SQL queries that utilize joins and subqueries you can provide a deeper and more granular level of analysis and data retrieval.

This assessment will continue looking at the familiar Northwind database that contains customer relationship management (CRM) data as well as employee and product data. You will take a deeper dive into this database in order to accomplish more advanced SQL queries that require you to access data from multiple tables at once. 

Imagine that you are working in an analyst role for the sales rep team. They have collaborated with the customer relations and the product teams to take a comprehensive look at the employee to customer pipeline in an attempt to find areas of improvement and potential growth. You have been asked to provide some specific data and statistics regarding this project.

## Learning Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Choose and perform whichever type of join is best for retrieving desired data
* Write subqueries to decompose complex queries

## Database

The database will be the customer relationship management (CRM) database, which has the following ERD.

![Database-Schema.png](ERD.png)

### Connect to the database

In the cell below we have provided the code to import both pandas and sqlite3 as well as define and create the connection to the database you will use. Also displayed is the schema and table names from the database. Use this information in conjunction with the ERD image above to assist in creating your SQL Queries.

Major Hint: Look for the shared columns across tables you need to 'join' together.

In [1]:
# CodeGrade step0
# Run this cell without changes

# SQL Library and Pandas Library
import sqlite3
import pandas as pd

# Connect to the database
conn = sqlite3.connect('data.sqlite')

pd.read_sql("""SELECT * FROM sqlite_master""", conn)

,type,name,tbl_name,rootpage,sql
0,table,orderdetails,orderdetails,2,"CREATE TABLE `orderdetails` (`orderNumber`, `p..."
1,table,payments,payments,28,"CREATE TABLE `payments` (`customerNumber`, `ch..."
2,table,offices,offices,32,"CREATE TABLE `offices` (`officeCode`, `city`, ..."
3,table,customers,customers,33,"CREATE TABLE `customers` (`customerNumber`, `c..."
4,table,orders,orders,38,"CREATE TABLE `orders` (`orderNumber`, `orderDa..."
5,table,productlines,productlines,46,"CREATE TABLE `productlines` (`productLine`, `t..."
6,table,products,products,47,"CREATE TABLE `products` (`productCode`, `produ..."
7,table,employees,employees,56,"CREATE TABLE `employees` (`employeeNumber`, `l..."


## Part 1: Join and Filter

### Step 1

The company would like to let Boston employees go remote but need to know more information about who is working in that office. Return the first and last names and the job titles for all employees in Boston.

In [2]:
# CodeGrade step1
# Replace None with your code
# We've queried the offices table for Boston and found the code is 2
df_boston = pd.read_sql("""
    SELECT firstName, lastName, jobTitle 
    FROM employees WHERE officeCode = '2';""", conn)
# Boston only has two employees, which is a bit sparse.

# This looks like a bad unit test, where it is expecting a shape of 2, 2 but the instructions clearly ask for 3 columns
# Confirmed, but as there is no obvious or elegant way to fix this, leaving as is
# The unit test is only checking firstName field, so theoretically we could leave either of the others out of the query
# This is bad practice

### Step 2

Recent downsizing and employee attrition have caused some mixups in office tracking and the company is worried they are supporting a 'ghost' location. Are there any offices that have zero employees?

In [3]:
# CodeGrade step2
# Replace None with your code
df_zero_emp = pd.read_sql("""
    SELECT * FROM offices
    LEFT JOIN employees on employees.officeCode = offices.officeCode
    WHERE employees.employeeNumber IS NULL;""",conn)
# Examining the data we find office codes go 1-7, and all of these appear in the employee officeCode list
# Empty results, no ghost offices

## Part 2: Type of Join

### Step 3

As a part of this larger analysis project the HR department is taking the time to audit employee records to make sure nothing is out of place and have asked you to produce a report of all employees. Return the employees first name and last name along with the city and state of the office that they work out of (if they have one). Include all employees and order them by their first name, then their last name.

In [4]:
# CodeGrade step3
# Replace None with your code
df_employee = pd.read_sql("""
    SELECT firstName, lastName, city, state FROM employees
    LEFT JOIN offices ON offices.officeCode = employees.officeCode
    ORDER BY firstName, lastName;""", conn)

### Step 4
The customer management and sales rep team know that they have several 'customers' in the system that have not placed any orders. They want to reach out to these customers with updated product catalogs to try and get them to place initial orders. Return all of the customer's contact information (first name, last name, and phone number) as well as their sales rep's employee number for any customer that has not placed an order. Sort the results alphabetically based on the contact's last name

There are several approaches you could take here, including a left join and filtering on null values or using a subquery to filter out customers who do have orders. In total there are 24 customers who have not placed an order.

In [5]:
# CodeGrade step4
# Replace None with your code
df_contacts = pd.read_sql("""
    SELECT contactFirstName, contactLastName, phone, salesRepEmployeeNumber FROM customers
    LEFT JOIN orders ON orders.customerNumber = customers.customerNumber
    WHERE orders.customerNumber IS NULL
    ORDER BY contactLastName;""", conn)

## Part 3: Built-in Function

### Step 5

The accounting team is auditing their figures and wants to make sure all customer payments are in alignment, they have asked you to produce a report of all the customer contacts (first and last names) along with details for each of the customers' payment amounts and date of payment. They have asked that these results be sorted in descending order by the payment amount.

Hint: A member of their team mentioned that they are not sure the 'amount' column is being stored as the right datatype so keep this in mind when sorting.

In [2]:
# CodeGrade step5
# Replace None with your code
df_payment = pd.read_sql("""
    SELECT contactFirstName, contactLastName, CAST(amount as REAL), paymentDate FROM customers
    INNER JOIN payments on payments.customerNumber = customers.customerNumber
    ORDER BY CAST(amount as REAL) DESC;""", conn)

# My first approach of a LEFT JOIN was bringing back payments with no customer info
# This was arguably incorrect because of our goal: "to make sure all customer payments are in alignment"
# Payments without customer info are suspicious though, and should be considered, possibly in a separate query

(273, 4)

## Part 4: Joining and Grouping

### Step 6

The sales rep team has noticed several key team members that stand out as having trustworthy business relations with their customers, reflected by high credit limits indicating more potential for orders. The team wants you to identify these 4 individuals. Return the employee number, first name, last name, and number of customers for employees whose customers have an average credit limit over 90k. Sort by number of customers from high to low.

In [5]:
# CodeGrade step6
# Replace None with your code
df_credit = pd.read_sql("""
    SELECT employeeNumber, firstName, lastName,
    COUNT(*) AS num_customers FROM employees
    INNER JOIN customers ON customers.salesRepEmployeeNumber = employees.employeeNumber
    GROUP BY employeeNumber
    HAVING AVG(creditLimit) > 90000
    ORDER BY COUNT(*) DESC;""", conn)
# We'll avoid using the alias in the having/Order by for portability
# We've removed the unrequested column "average_credit_limit"
# We've updated to an INNER JOIN as this correctly reflects the necesseity of matching employee numbers
# In this specific case this does not change the results

### Step 7

The product team is looking to create new model kits and wants to know which current products are selling the most in order to get an idea of what is popular. Return the product name and count the number of orders for each product as a column named 'numorders'. Also return a new column, 'totalunits', that sums up the total quantity of product sold (use the quantityOrdered column). Sort the results by the totalunits column, highest to lowest, to showcase the top selling products.

In [8]:
# CodeGrade step7
# Replace None with your code
df_product_sold = pd.read_sql("""
    SELECT productName, SUM(quantityOrdered) AS totalunits, COUNT(DISTINCT orderNumber) AS numorders FROM products
    INNER JOIN orderdetails ON orderdetails.productCode = products.productCode
    GROUP BY productName
    ORDER BY totalunits DESC;""", conn)
# Similar to step 6, changed to an INNER JOIN
# If we cannot connect the order info to the products table, this will not help us evaluate products
# This reduces the number of rows by 1 (in this case)

(109, 3)

## Part 5: Multiple Joins

### Step 8

As a follow-up to the above question, the product team also wants to know how many different customers ordered each product to get an idea of market reach. Return the product name, code, and the total number of customers who have ordered each product, aliased as 'numpurchasers'. Sort the results by the highest  number of purchasers.

Hint: You might need to join more than 2 tables. Use DISTINCT to return unique/different values.

In [9]:
# CodeGrade step8
# Replace None with your code
df_total_customers = pd.read_sql("""
    SELECT productName, products.productCode, 
    COUNT(DISTINCT customerNumber) AS numpurchasers FROM products
    INNER JOIN orderdetails ON orderdetails.productCode = products.productCode
    INNER JOIN orders ON orders.orderNumber = orderdetails.orderNumber
    GROUP BY productName
    ORDER BY numpurchasers DESC;""", conn)
# Similar to steps 6 + 7. We're connecting products to customers, so INNER JOIN correctly achieves this

(109, 3)

### Step 9

The custom relations team is worried they are not staffing locations properly to account for customer volume. They want to know how many customers there are per office. Return the count as a column named 'n_customers'. Also return the office code and city.

In [10]:
# CodeGrade step9
# Replace None with your code
df_customers = pd.read_sql("""
    SELECT offices.city, offices.officeCode,
    COUNT(DISTINCT customerNumber) as n_customers FROM offices
    LEFT JOIN employees ON employees.officeCode = offices.officeCode
    LEFT JOIN customers ON customers.salesRepEmployeeNumber = employees.employeeNumber
    GROUP BY offices.city;""", conn)
# We were not asked to do any sorting

## Part 6: Subquery

### Step 10

Having looked at the results from above, the product team is curious to dig into the underperforming products. They want to ask members of the team who have sold these products about what kind of messaging was successful in getting a customer to buy these specific products. Using a subquery or common table expression (CTE), select the employee number, first name, last name, city of the office, and the office code for employees who sold products that have been ordered by fewer than 20 customers.

Hint: Start with the subquery, find all the products that have been ordered by 19 or less customers, consider adapting one of your previous queries.

In [9]:
# CodeGrade step10
# Replace None with your code
# We're using our approach from Step 8 as a subquery (with slight tweaks)
df_under_20 = pd.read_sql("""
    SELECT DISTINCT employees.employeeNumber, employees.firstName, employees.lastName, offices.city, offices.officeCode
    FROM employees
    INNER JOIN customers ON customers.salesRepEmployeeNumber = employees.employeeNumber
    INNER JOIN orders ON orders.customerNumber = customers.customerNumber
    INNER JOIN orderdetails ON orderdetails.orderNumber = orders.orderNumber
    INNER JOIN offices ON offices.officeCode = employees.officeCode
    WHERE orderdetails.productCode IN (
        SELECT products.productCode
        FROM products
        INNER JOIN orderdetails ON orderdetails.productCode = products.productCode
        INNER JOIN orders ON orders.orderNumber = orderdetails.orderNumber
        GROUP BY products.productCode
        HAVING COUNT(DISTINCT customerNumber) < 20
    )
    ORDER BY employees.lastName;""", conn)

# We've added the ORDER BY because this is expected in the unit tests
# There is no mention of a need to sort/order in

,employeeNumber,firstName,lastName,city,officeCode
0,1337,Loui,Bondur,Paris,4
1,1501,Larry,Bott,London,7
2,1401,Pamela,Castillo,Paris,4
3,1188,Julie,Firrelli,Boston,2
4,1611,Andy,Fixter,Sydney,6
5,1702,Martin,Gerard,Paris,4
6,1370,Gerard,Hernandez,Paris,4
7,1165,Leslie,Jennings,San Francisco,1
8,1504,Barry,Jones,London,7
9,1612,Peter,Marsh,Sydney,6


### Close the connection

In [12]:
# Run this cell without changes

conn.close()